# Desarrollo de la Red Neuronal

### Arquitectura de la Red

<img src= "Red_Neuronal.png">

Como podemos observar en el grafico anterior, la red tendra **2 capas ocultas y 1 de salida**.

En la **primer capa oculta** contaremos con **8 neuronas**, en la **segunda capa oculta** tendremos **4 neuronas** y en la **capa de salida tendremos 1 neurona** que nos indicara si el hongo es **comestible o no**.

Para las **capas ocultas** utilizaremos la **función de activación Relu** ya que convierte los valores negativos en 0 y mitiga el proble,a del gradiente desvaneciente.
Por otro lado, para la **capa de salida** utilizaremos la **función de activación Logística** ya que devuelve valores entre 0 y 1 y eso es ideal para nuestra red.


### Implementacion en numpy de la Red

**Forward Propagation**

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

all_data = pd.read_csv("mushrooms_limpios.csv") #22418 (Cantidad de registros)

all_inputs = all_data.iloc[:, 1:].values # Selecciono todas las columnas de entrada del conjunto de datos, menos la primera que es la de salida
all_outputs = all_data.iloc[:, 0].values # Selecciono la columna de salida del conjunto de datos

scaler = StandardScaler() # Escalo los datos de entrada 
all_inputs = scaler.fit_transform(all_inputs) # Transformo las entradas para tener una media de 0 y una desviación estándar de 1

X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1 / 3) # Divido los conjuntos de datos de entrenamiento y prueba
n = X_train.shape[0] 

# Defino funciones de activación
relu = lambda x: np.maximum(x, 0)  # ReLU porque no tengo valores negativos
logistic = lambda x: 1 / (1 + np.exp(-x)) # Logística para la capa de salida (salida binaria) 

np.random.seed(22) # Inicializo semilla en 22 para reproducibilidad

w_hidden = np.random.rand(8, 11)   # Pesos primera capa oculta
w_hidden2 = np.random.rand(4, 8)   # Pesos segunda capa oculta        
w_output = np.random.rand(1, 4)    # Pesos capa de salida

b_hidden = np.random.rand(8, 1)    # Sesgos primera capa oculta   
b_hidden2 = np.random.rand(4, 1)   # Sesgos segunda capa oculta
b_output = np.random.rand(1, 1)    # Sesgos capa de salida

# Función de forward propagation
def forward_prop(X): 
    Z1 = w_hidden @ X + b_hidden       # Entrada ponderada de la primera capa oculta
    A1 = relu(Z1)                      # ReLU en la primera capa oculta
    Z2 = w_hidden2 @ A1 + b_hidden2    # Entrada ponderada de la segunda capa oculta
    A2 = relu(Z2)                      # ReLU en la segunda capa oculta
    Z3 = w_output @ A2 + b_output      # Entrada ponderada de la capa de salida
    A3 = logistic(Z3)                  # Logistica en la capa de salida
    return Z1, A1, Z2, A2, Z3, A3

Precisión No entrenada

In [36]:
test_predictions = forward_prop(X_test.transpose())[5]  # Consulto A3 (Capa de salida)
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test) # Comparo valores predichos con reales
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0]) # Calculo la precisión

print(accuracy)

0.5329854141576078


### Entrenamiento y Evaluacion

**Back Propagation**

In [37]:
L = 0.05 # Tasa de aprendizaje

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Función de backward propagation
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y): 
    # Calculo de las derivadas para obtener la derivada del costo con respecto a mis pesos y sesgos (W3, W2, W1, B3, B2 B1)
    dC_dA3 = 2 * (A3 - Y)           
    dA3_dZ3 = d_logistic(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3   

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = d_relu(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    # Calculo los gradientes de la función de costo con respecto a los pesos y sesgos de cada capa.
    # keepdims sirve para mantener las dimensiones de entrada
    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)  

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)
    
    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3


#### Descenso de Gradiente Estocástico

In [38]:
for i in range(120_000):
    # Selecciono aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # Los paso aleatoriamente por la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # Realizo retropropagación y devuelvo los pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # Actualizo pesos y sesgos con la taza de aprendizaje 
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

Precisión Red Entrenada

In [39]:
# Calculo de precisión de entrenamiento
train_predictions = forward_prop(X_train.transpose())[5]  # Consulto A3 (Capa de salida)
train_comparisons = np.equal((train_predictions >= .5).flatten().astype(int), Y_train) # Comparo valores predichos con reales
train_accuracy = sum(train_comparisons.astype(int)) / X_train.shape[0] # Calculo la precisión de entrenamiento

print(f"Train Accuracy: {train_accuracy}")
    
    
# Calculo de precisión de prueba
test_predictions = forward_prop(X_test.transpose())[5]  # Consulto A3 (Capa de salida)
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test) # Comparo valores predichos con reales
test_accuracy = sum(test_comparisons.astype(int)) / X_test.shape[0] # Calculo la precisión de prueba

print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.8966878554700569
Test Accuracy: 0.8837147062759266


<img src= "mi_red_iteraciones=150000_L=0.05.png">

Como podemos observar en el grafico anterior, cuando alcanzamos las 120000 iteraciones aproximadamente, conseguimos la menor variacion del resultado y la menor diferencia entre test y train.

# Comparación con scikit-learn

### Scikit-Learn

In [40]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

df = pd.read_csv("mushrooms_limpios.csv")

 
X = df.values[:, 1:] # Selecciono todas las columnas de entrada del conjunto de datos, menos la primera que es la de salida
scaler = StandardScaler() # Escalo los datos de entrada
X  = scaler.fit_transform(X)

Y = df.values[:, 0] # Selecciono la columna de salida del conjunto de datos

# Separar los datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3)

nn = MLPClassifier(solver='adam',
                   hidden_layer_sizes=(8, 4, ),
                   activation='relu',
                   max_iter=120_000,
                   random_state=22,
                   learning_rate_init=.05)

nn.fit(X_train, Y_train)

print("Puntaje de entrenamiento: %f" % nn.score(X_train, Y_train))
print("Puntaje de prueba: %f" % nn.score(X_test, Y_test))


Puntaje de entrenamiento: 0.891134
Puntaje de prueba: 0.894955


Los resultados son casi iguales, estando un poco por encima los resultados de la libreria, que es normal debido a que tiene una mejor y mas óptima implementación.

Claramente notamos diferencia sobre todo en el tiempo de ejecucion ya que mi red tarda casi 1 minuto en ejecutar el descenso de gradiente estocastico y la de scikit-learn tarda menos de 2 segundos. Esto se debe a que la libreria scikit-learn utiliza metodos de optimizacion que le permiten ejecutarse mucho mas rápido.

# Conclusión Final


Implementar una red neuronal para clasificación binaria desde cero me ha proporcionado una comprensión mas profunda del funcionamiento de las mismas. 

A lo largo del proyecto, he aprendido a seleccionar un Data Frame, a entenderlo, a manipularlo de manera que me sirva para desarrollar mi red. Ademas, aprendí a limpiarlo de valores atipicos y vacíos, y a elegir cuando esos valores me sirven o me perjudican.

Luego, aprendi a desarrollar paso a paso mi red a lo largo del desarrollo, cambiando la cantidad de iteraciones, la taza de aprendizaje y analizando sus graficos para elegir los mejores valores posibles.

Al comparar esta experiencia con el uso de librerías como scikit-learn, es evidente que construir una red neuronal desde cero ofrece una mayor comprensión del funcionamiento de la misma y una mayor personalización. Sin embargo, el tiempo de desarrollo y la probabilidad de errores son mucho mayores. 
Teniendo esto en cuenta, usar la libreria, es mucho mas práctico para cuando la necesitamos para un proyecto simple o rápido y que no merezca mucho analisis, ademas, con la libreria podemos comparar distintos metodos de aprendizaje que pueden implementarse ante el mismo problema.

En conclusión, mi opinión es que desarrollar tu propia Red te ayuda mucho mas a entender realmente su funcionamiento, su poder y sus limitaciones.